In [2]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hhs1516","key":"df77d1870d62b8065b1ea188acc8273c"}'}

In [3]:
ls -1ha kaggle.json

kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c lish-moa

  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 65.3MB/s]
 78% 43.0M/54.8M [00:00<00:00, 43.3MB/s]
100% 54.8M/54.8M [00:00<00:00, 80.3MB/s]
  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 35.7MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 177MB/s]
  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 84.4MB/s]
 55% 5.00M/9.16M [00:00<00:00, 23.9MB/s]
100% 9.16M/9.16M [00:00<00:00, 35.6MB/s]


In [6]:
!unzip -n /content/test_features.csv.zip
!unzip -n /content/train_features.csv.zip
!unzip -n /content/train_targets_nonscored.csv.zip
!unzip -n /content/train_targets_scored.csv.zip
!unzip -n /content/sample_submission.csv.zip

Archive:  /content/test_features.csv.zip
  inflating: test_features.csv       
Archive:  /content/train_features.csv.zip
  inflating: train_features.csv      
Archive:  /content/train_targets_nonscored.csv.zip
  inflating: train_targets_nonscored.csv  
Archive:  /content/train_targets_scored.csv.zip
  inflating: train_targets_scored.csv  
Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [8]:
train_features = pd.read_csv("/content/train_features.csv")
train_targets = pd.read_csv("/content/train_targets_scored.csv")
test_features = pd.read_csv("/content/test_features.csv")
submission = pd.read_csv("/content/sample_submission.csv")

In [65]:
COLS = ['cp_type','cp_dose']
FE = []
for col in COLS:
    for mod in train_features[col].unique():
        train_features[mod] = (train_features[col] == mod).astype(int)
del train_features['sig_id']
del train_features['cp_type']
del train_features['cp_dose']
FE+=list(train_features.columns) 
del train_targets['sig_id']

In [10]:
non_ctl_idx = train_features.loc[train_features['cp_type']!='ctl_vehicle'].index.to_list()
train_features = train_features.drop(['sig_id','cp_type','cp_dose','cp_time'],axis=1)

In [11]:
train_features

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,0.2965,-0.5055,-0.5119,-0.2162,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,0.1656,0.5300,-0.2568,-0.0455,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,0.1256,-0.1219,5.4470,1.0310,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,-1.6720,-1.2690,3.0900,-0.3814,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,-1.4470,1.0620,0.7888,-0.0848,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.6456,0.0136,-0.5

In [67]:
def model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(len(FE)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(4096, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(4096, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(206, activation="sigmoid")
        ])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2.75e-5), 
                  loss='binary_crossentropy', metrics=["accuracy", "AUC"])
    return model

In [71]:
sum_model = model()

In [72]:
sum_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_48 (Batc (None, 877)               3508      
_________________________________________________________________
dense_48 (Dense)             (None, 4096)              3596288   
_________________________________________________________________
batch_normalization_49 (Batc (None, 4096)              16384     
_________________________________________________________________
dropout_32 (Dropout)         (None, 4096)              0         
_________________________________________________________________
dense_49 (Dense)             (None, 4096)              16781312  
_________________________________________________________________
batch_normalization_50 (Batc (None, 4096)              16384     
_________________________________________________________________
dropout_33 (Dropout)         (None, 4096)            

In [68]:
from sklearn.model_selection import KFold
NFOLD = 5
kf = KFold(n_splits=NFOLD)

BATCH_SIZE=128
EPOCHS=35

for col in COLS:
    for mod in test_features[col].unique():
        test_features[mod] = (test_features[col] == mod).astype(int)
sig_id = pd.DataFrame()
sig_id = test_features.pop('sig_id')
del test_features['cp_type']
del test_features['cp_dose']

pe = np.zeros((test_features.shape[0], 206))

train_features = train_features.values
train_targets = train_targets.values
pred = np.zeros((train_features.shape[0], 206))

cnt=0
for tr_idx, val_idx in kf.split(train_features):
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                       patience=3, verbose=1, epsilon=1e-4, mode='min')
    cnt += 1
    print(f"FOLD {cnt}")
    net = model()
    net.fit(train_features[tr_idx], train_targets[tr_idx], batch_size=BATCH_SIZE,
            epochs=EPOCHS, 
            validation_data=(train_features[val_idx], train_targets[val_idx]),
            verbose=0, callbacks=[reduce_lr_loss])
    print("train", net.evaluate(train_features[tr_idx], train_targets[tr_idx]
                                , verbose=0, batch_size=BATCH_SIZE))
    print("val", net.evaluate(train_features[val_idx], train_targets[val_idx], 
                              verbose=0, batch_size=BATCH_SIZE))
    print("predict val...")
    pred[val_idx] = net.predict(train_features[val_idx], batch_size=BATCH_SIZE,
                                verbose=0)
    print("predict test...")
    pe += net.predict(test_features, batch_size=BATCH_SIZE, verbose=0) / NFOLD

FOLD 1
train [0.015100102871656418, 0.21054013073444366, 0.9484171867370605]
val [0.01728702336549759, 0.11757295578718185, 0.8342134356498718]
predict val...
predict test...
FOLD 2
train [0.01495895255357027, 0.21489685773849487, 0.9493069648742676]
val [0.017290450632572174, 0.13352929055690765, 0.8176873922348022]
predict val...
predict test...
FOLD 3
train [0.014953039586544037, 0.21075008809566498, 0.9506030678749084]
val [0.017102297395467758, 0.13247953355312347, 0.8246291279792786]
predict val...
predict test...
FOLD 4
train [0.014835589565336704, 0.2079680860042572, 0.949800431728363]
val [0.01718057133257389, 0.1259710192680359, 0.8171308636665344]
predict val...
predict test...
FOLD 5
train [0.01484842598438263, 0.20491287112236023, 0.9486188888549805]
val [0.017370807006955147, 0.1228475421667099, 0.8247319459915161]
predict val...
predict test...


In [82]:
prediction = net.predict(test_features, batch_size=BATCH_SIZE, verbose=0)
prediction

array([[0.00398982, 0.00301532, 0.00308854, ..., 0.00554144, 0.00360243,
        0.00426264],
       [0.00492084, 0.0035912 , 0.00415102, ..., 0.0028581 , 0.00762584,
        0.00408813],
       [0.00274051, 0.00218062, 0.00258849, ..., 0.00228282, 0.00389753,
        0.00302504],
       ...,
       [0.00351676, 0.00269745, 0.0036067 , ..., 0.0027693 , 0.00257092,
        0.00288143],
       [0.00472183, 0.00305882, 0.00304105, ..., 0.00394923, 0.00254885,
        0.00589463],
       [0.00347873, 0.00240814, 0.00391362, ..., 0.00708447, 0.00418116,
        0.00405556]], dtype=float32)